In [1]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random


path ='/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt'
# path = '/Users/amberedmundson/Laboratory/clones/word2vec/text.txt'

v = nlp.Vocab()
t = nlp.Text(path,v)

v2 = nlp.Vocab(t.vocab)
v2.sort(10)

t.ChangeVocab(v2)
v2.InitUnigramTable()

vocab_size = v2.size
dim = 32

seed = la.Seed(1)

syn0 = nn.FlexLinear(vocab_size,dim)

syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= dim

syn1neg = nn.FlexLinear(dim,vocab_size)
syn1neg.weights.zero()

layers = list()
layers.append(syn0)
layers.append(syn1neg)
layers.append(nn.FlexSigmoid(vocab_size))

mlp = nn.FlexSequential(layers)

criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)
sampler = nn.Sampler(v2,5)
cbow = nn.CBOW(t.sentences,v2,sampler,5,5,False)

In [ ]:
%%timeit
optim.train(cbow,0.01,1,6)

In [9]:
import numpy as np

word2vec = {}
vocab = list()
for i in range(v2.size):
    l2 = np.sqrt(syn0.weights[i].dot(syn0.weights[i]))
    v = syn0.weights[i]
    v /= l2
    vocab.append(v2.getTermAtIndex(i).letters)
    word2vec[v2.getTermAtIndex(i).letters] = syn0.weights[i]
    
sim = Counter()
for word in vocab:
    sim[word] = word2vec['ron'].dot(word2vec[word])
sim.most_common()

[('ron', 1.0000001192092896),
 ('hermione', 0.9807183742523193),
 ('hagrid', 0.9490734338760376),
 ('malfoy', 0.9455265402793884),
 ('dumbledore', 0.939634382724762),
 ('snape', 0.933041512966156),
 ('mcgonagall', 0.9289185404777527),
 ('lupin', 0.9207931756973267),
 ('weasley', 0.9167242050170898),
 ('vernon', 0.9056099653244019),
 ('harry', 0.9014864563941956),
 ('ginny', 0.8975412249565125),
 ('he', 0.8953753113746643),
 ('neville', 0.8899844288825989),
 ('sirius', 0.879888117313385),
 ('she', 0.8784891963005066),
 ('umbridge', 0.8691948056221008),
 ('petunia', 0.8674612045288086),
 ('fudge', 0.8635996580123901),
 ('diggory', 0.8617649078369141),
 ('voldemort', 0.8608818054199219),
 ('dudley', 0.8579322099685669),
 ('moody', 0.857856810092926),
 ('george', 0.8555391430854797),
 ('hooch', 0.8455556631088257),
 ('yaxley', 0.8420859575271606),
 ('maxime', 0.8405681252479553),
 ('pomfrey', 0.8392345309257507),
 ('karkaroff', 0.8365315794944763),
 ('wood', 0.8320019245147705),
 ('fred', 